# RGB and CIE
In this project we'll regularly need to calculate distances between pixels in colour space as a proxy for the visual difference between the colours. The simplest way of doing this is to calculate the cosine/euclidian/l2 distance, as written below in maths and python:

${\displaystyle \Delta C ={\sqrt {(R_{2}-R_{1})^{2}+(G_{2}-G_{1})^{2}+(B_{2}-B_{1})^{2}}}}$

In [ ]:
def colour_distance(colour_1, colour_2):
    return sum([(a - b) ** 2 for a, b in zip(colour_1, colour_2)]) ** 0.5

The red, green, and blue channels available to us in RGB space are ideally suited for representing colour on pixelated screens. However, our goal is to represent the _differences_ between colours, and RGB isn't ideal for this. It's now [pretty well established](https://en.wikipedia.org/wiki/Color_difference) that euclidian distances in RGB space are a bad representation of the distances that our eyes see.
By stretching the RGB dimensions by different amounts, we can better approximate that difference:

${\displaystyle {\sqrt {2\times \Delta R^{2}+4\times \Delta G^{2}+3\times \Delta B^{2}}}}$

Again, here's the python

In [ ]:
def colour_distance(colour_1, colour_2):
    r_1, g_1, b_1 = colour_1
    r_2, g_2, b_2 = colour_2

    return (2 * (r_1 - r_2) ** 2 +
            4 * (g_1 - g_2) ** 2 +
            3 * (b_1 - b_2) ** 2) ** 0.5

We can improve further by adding some extra weirdness to the red and blue channels at the end

${\displaystyle \Delta C={\sqrt {2\times \Delta R^{2}+4\times \Delta G^{2}+3\times \Delta B^{2}+{{{\bar {r}}\times (\Delta R^{2}-\Delta B^{2})} \over {256}}}}}$

Here it is in python

In [ ]:
def colour_distance(colour_1, colour_2):
    r_1, g_1, b_1 = colour_1
    r_2, g_2, b_2 = colour_2
    
    d_r_sq = (r_1 - r_2) ** 2
    d_g_sq = (g_1 - g_2) ** 2
    d_b_sq = (b_1 - b_2) ** 2
    mean_r = (r_1 + r_2) / 2
    
    d_c_sq = (2 * d_r_sq + 4 * d_g_sq + 3 * d_b_sq +
              (mean_r * (d_r_sq - d_b_sq) / 256))

    return d_c_sq ** 0.5

The most general and efficient approach (as far as I know) is to transform the entire image's RGB coordinates into a new space. The CIE produced [CIELAB](https://en.wikipedia.org/wiki/CIELAB_color_space#CIELAB) to better approximate human perception. The axes of the space are as follows

> The three coordinates of CIELAB represent the lightness of the color (`L` = 0 yields black and `L` = 100 indicates diffuse white; specular white may be higher), its position between red/magenta and green (`a`, negative values indicate green while positive values indicate magenta) and its position between yellow and blue (`b`, negative values indicate blue and positive values indicate yellow).

[CIE76](https://en.wikipedia.org/wiki/Color_difference#CIE76) (ie euclidian distance in LAB space) was the original distance proposed with the space. It's been improved upon since, but the differences are minor and as far as I've seen, are unnecessary complications for such minimal gain.

Implementing the map from RGB to LAB and back is incredibly simple - all we need to do is import the relevant function from `skimage`

In [ ]:
from skimage.color import rgb2lab, lab2rgb

We can now use our original, super-simple colour distance!